# Run all train, all test and upload

In [1]:
import time
import numpy as np
from tools.csp import generate_projection, generate_eye, extract_feature
from tools.filters import load_filterbank
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from tools.data import DreemDatasets
from preprocessing import Compose, ExtractBands, ExtractSpectrum
from models.riemannian_multiscale import riemannian_multiscale
import pandas as pd

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
fs = 50.  # sampling frequency
NO_channels = 7  # number of EEG channels
NO_riem = int(NO_channels * NO_channels + 1) / 2  # Total number of CSP feature per band and timewindow
bw = np.array([2, 4, 8, 13, 22])
ftype = 'butter'  # 'fir', 'butter'
forder = 2  # 4
filter_bank = load_filterbank(bw, fs, order=forder, max_freq=23, ftype=ftype)  # get filterbank coeffs
time_windows_flt = np.array([[0, 30],
                            [5, 25],
                             [15, 30],
                             [10, 25],
                             [5, 20],
                             [5, 15],
                             [0, 10],
                             [5, 15],
                             [15, 25],
                             [10, 20],
                             [5, 15],
                             [5, 10]]) * fs
time_windows = time_windows_flt.astype(int)
# restrict time windows and frequency bands
#time_windows = time_windows[0:1]  # use only largest timewindow

NO_bands = filter_bank.shape[0]
riem_opt = "No_Adaptation"  # {"Riemann","Riemann_Euclid","Whitened_Euclid","No_Adaptation"}
rho = 0.1
NO_csp = 20  # Total number of CSP feature per band and timewindow
useCSP = True

Todo : 

    1- refaire l'embedding du test avc le train split + résultats sans extra data
    2- refaire résultats du test avc le all train sans extra data
    3- résultats du 2 avec extra data

In [3]:
def get_data(path, train=True):
    if train:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/train/eeg_" + str(i + 1) + ".npy")
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
                X[0] = feature_0
                del feature_0
            else:
                X[i] = np.load("dataset/"+path+"/train/eeg_" + str(i + 1) + ".npy")
        Y = np.load("dataset/"+path+"/train/targets.npy")
        X = X.transpose((1, 0, 2))
        return(X, Y)
    else:
        for i in range(7):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/test/eeg_" + str(i + 1) + ".npy")
                X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
                X[0] = feature_0
                del feature_0
            else:
                X[i] = np.load("dataset/"+path+"/test/eeg_" + str(i + 1) + ".npy")
        X = X.transpose((1, 0, 2))
        return(X)

def get_extra_data(path, train=True):
    if train:
        use_datasets = ["accelerometer_x","accelerometer_y","accelerometer_z","pulse_oximeter_infrared"]
        for i in range(4):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/train_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                print(feature_0.shape)
                X = np.zeros((4, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
            else:
                feature_0 = np.load("dataset/"+path+"/train_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
        Y = np.load("dataset/"+path+"/train_split/targets.npy")
        X = X.transpose((1, 0, 2))
        return(X, Y)
    else:
        use_datasets = ["accelerometer_x","accelerometer_y","accelerometer_z","pulse_oximeter_infrared"]
        for i in range(4):
            if i==0:
                feature_0 = np.load("dataset/"+path+"/test/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                print(feature_0.shape)
                X = np.zeros((4, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
            else:
                feature_0 = np.load("dataset/"+path+"/test/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
                X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
                del feature_0
        X = X.transpose((1, 0, 2))
        return(X)
    
def get_data_val(path):
    for i in range(7):
        if i==0:
            feature_0 = np.load("dataset/"+path+"/val_split/eeg_" + str(i + 1) + ".npy")
            X = np.zeros((7, feature_0.shape[0], feature_0.shape[1]))
            X[0] = feature_0
            del feature_0
        else:
            X[i] = np.load("dataset/"+path+"/val_split/eeg_" + str(i + 1) + ".npy")
    Y = np.load("dataset/"+path+"/val_split/targets.npy")
    X = X.transpose((1, 0, 2))
    return(X, Y)

def get_extra_data_val(path):
    use_datasets = ["accelerometer_x","accelerometer_y","accelerometer_z","pulse_oximeter_infrared"]
    for i in range(4):
        if i==0:
            feature_0 = np.load("dataset/"+path+"/val_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
            print(feature_0.shape)
            X = np.zeros((4, feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2]))
            X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
            del feature_0
        else:
            feature_0 = np.load("dataset/"+path+"/val_split/" + use_datasets[i] + ".npy").transpose((1, 0, 2))
            X[i] = feature_0.reshape(feature_0.shape[0], feature_0.shape[1]*feature_0.shape[2])
            del feature_0
    Y = np.load("dataset/"+path+"/val_split/targets.npy")
    X = X.transpose((1, 0, 2))
    return(X, Y)

In [4]:
def get_features(data_train,data_test, label, useCSP = True, NO_csp = 20):
    if useCSP:
        w = generate_projection(data_train, label, NO_csp, filter_bank, time_windows, NO_classes=5)
    else:
        w = generate_eye(data_train, label, filter_bank, time_windows)
    #feature_train = extract_feature(data_train, w, filter_bank, time_windows)
    feature_test = extract_feature(data_test, w, filter_bank, time_windows)
    return(w, feature_test)

## Get all features 

In [5]:
path = "all"

train_data, train_label = get_data(path, train = True)
test_data = get_data(path, train = False)


useCSP = True
start = time.time()
w, test_feat_CSP = get_features(train_data, test_data, train_label, useCSP)
del w
np.save("features_CSP_test_with_train_split_True.npy", test_feat_CSP)
print(time.time()-start)
print("end CSP true")
del test_feat_CSP
"""
useCSP = False
start = time.time()
w, train_feat_CSP, test_feat_CSP = get_features(train_data, test_data, train_label, useCSP)
np.save("features_CSP_train_False.npy", train_feat_CSP)
np.save("features_CSP_test_False.npy", test_feat_CSP)
print(time.time()-start)
print("end CSP False")

del w
del train_feat_CSP
del test_feat_CSP

methods = ["No_Adaptation","Whitened_Euclid"]
for riem_opt in methods:
    try:
        start = time.time()
        riemann = riemannian_multiscale(filter_bank, time_windows, riem_opt=riem_opt, rho=rho, vectorized=True)
        train_feat_R = riemann.fit(train_data)
        test_feature_R = riemann.features(test_data)
        np.save("features_R_train_"+str(riem_opt), train_feat_CSP)
        np.save("features_R_test_"+str(riem_opt), test_feature_R)
        print(time.time()-start)
        print(riem_opt)
        del train_feat_R
        del test_feature_R
    except Exception as e:
        print(e)
        del train_feat_R
        del test_feature_R
        pass
"""

3581.9479615688324
end CSP true


'\nuseCSP = False\nstart = time.time()\nw, train_feat_CSP, test_feat_CSP = get_features(train_data, test_data, train_label, useCSP)\nnp.save("features_CSP_train_False.npy", train_feat_CSP)\nnp.save("features_CSP_test_False.npy", test_feat_CSP)\nprint(time.time()-start)\nprint("end CSP False")\n\ndel w\ndel train_feat_CSP\ndel test_feat_CSP\n\nmethods = ["No_Adaptation","Whitened_Euclid"]\nfor riem_opt in methods:\n    try:\n        start = time.time()\n        riemann = riemannian_multiscale(filter_bank, time_windows, riem_opt=riem_opt, rho=rho, vectorized=True)\n        train_feat_R = riemann.fit(train_data)\n        test_feature_R = riemann.features(test_data)\n        np.save("features_R_train_"+str(riem_opt), train_feat_CSP)\n        np.save("features_R_test_"+str(riem_opt), test_feature_R)\n        print(time.time()-start)\n        print(riem_opt)\n        del train_feat_R\n        del test_feature_R\n    except Exception as e:\n        print(e)\n        del train_feat_R\n        

In [5]:
path = "all"
#_, train_label = get_data(path, train = True)
train_extra_data, train_extra_label = get_extra_data(path, train = True)
val_extra_data, val_extra_label = get_extra_data_val(path)
test_extra_data = get_extra_data(path, train = False)
train_extra_data = train_extra_data.reshape(-1, 4*16)
val_extra_data = val_extra_data.reshape(-1, 4*16)
test_extra_data = test_extra_data.reshape(-1,  4*16)

(30631, 4, 4)
(7658, 4, 4)
(37439, 4, 4)


In [6]:
#_, train_label = get_data(path, train = True)
features_CSP_train = np.load("dataset/all/features_CSP_train_split_True.npy")
features_CSP_val = np.load("dataset/all/features_CSP_val_split_True.npy")
features_CSP_test = np.load("features_CSP_test_with_train_split_True.npy")

all_final_features = features_CSP_train
all_final_features_test = features_CSP_test
all_final_features_val = features_CSP_val

#all_final_features = np.concatenate((features_CSP_train, train_extra_data), axis= 1)
#all_final_features_test = np.concatenate((features_CSP_test, test_extra_data), axis= 1)
#all_final_features_val = np.concatenate((features_CSP_val, val_extra_data), axis= 1)
print(all_final_features.shape)
print(all_final_features_test.shape)

(30631, 4320)
(37439, 4320)


In [7]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(all_final_features, train_extra_label)
print("trained")
labels_pred = clf.predict(all_final_features_val)
print("predicted")

trained
predicted


In [8]:
CM = confusion_matrix(val_extra_label, labels_pred)
Acc = accuracy_score(val_extra_label, labels_pred)
F1 = f1_score(val_extra_label, labels_pred, average='macro')

print(CM, Acc, F1)

[[ 495   10  154    5   71]
 [  40   11  147    0   62]
 [  55    5 2935   94  348]
 [  15    1  437  702   31]
 [  57    7  558   14 1404]] 0.7243405588926612 0.5920934903258285


In [9]:
labels_pred = clf.predict(all_final_features_test)
print("predicted")

df = pd.DataFrame(labels_pred)
df.to_csv('label_pred2.csv', header=["sleep_stage"])
print("done")

predicted
done


In [12]:
all_final_features = np.concatenate((features_CSP_train, train_extra_data), axis= 1)
all_final_features_test = np.concatenate((features_CSP_test, test_extra_data), axis= 1)
all_final_features_val = np.concatenate((features_CSP_val, val_extra_data), axis= 1)
print(all_final_features.shape)
print(all_final_features_test.shape)


clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(all_final_features, train_extra_label)
print("trained")
labels_pred = clf.predict(all_final_features_val)
print("predicted")

CM = confusion_matrix(val_extra_label, labels_pred)
Acc = accuracy_score(val_extra_label, labels_pred)
F1 = f1_score(val_extra_label, labels_pred, average='macro')

print(CM, Acc, F1)

(30631, 4384)
(37439, 4384)
trained
predicted
[[ 525    9  145    1   55]
 [  54   20  124    1   61]
 [  65    4 2969   89  310]
 [  18    1  419  732   16]
 [  54   11  555   12 1408]] 0.7383128754243928 0.6158796718010775


In [13]:
labels_pred = clf.predict(all_final_features_test)
print("predicted")

predicted


In [14]:
df = pd.DataFrame(labels_pred)
df.to_csv('label_pred.csv', header=["sleep_stage"])
print("done")

done
